# Ollaman käyttö

-Valtteri Airaksinen

---

### Virtuaaliympäristön luonti ja käyttö

Virtuaaliympäristön luonti:

`python -m venv llama`

Ympäristön käyttöönotto:

`source llama/Scripts/activate`

Pakettien lataaminen:

`pip install -r requirements.txt`

Ympäristön käytöstä poisto:

`deactivate`

---

### Ollaman käyttö

Käynnistys:

`ollama serve`

Haetaan mallit:

Avaa uusi terminaali -> Ota ympäristö käyttöön

Nyt voit valita minkä mallin haluat ja noutaa sen esimerkiksi komennolla `ollama pull llama3.2:3b`

Malleja:

`llama3.2:1b` - 1.3GB

`llama3.2:3b` - 2GB

`llama3.1:8b` - 4.7GB

`llama3.1:70b` - 40GB

`llama3.1:405b` - 229GB

`llama3:8b` - 4.7GB

`llama3:70b` - 40GB

Jos mallia ei spesifioi, noudetaan tuorein malli esim `llama3.2` -> `llama3.2:3b`

Näytä ladatut mallit:

`ollama list`

---

In [ ]:
from langchain_community.llms import Ollama

# Specify which model you want to use
llm = Ollama(model="llama3:latest")

# Chatbot
print(llm.invoke("Ymmärrätkö jos annan sinulle käsiteltäväksi suomenkielistä dataa jonka haluan sinun jaoittelevan?", stop=['<|eot_id|>']))


### Suomenkielen ymmärrys

Kysymys: Ymmärrätkö jos annan sinulle käsiteltäväksi suomenkielistä dataa jonka haluan sinun jaoittelevan?

---

`llama3.2:1b` Ehko, en ole yhtynyt kyseessä olevaan suomen kielen ja taidetta koskevaan tuotannoon. Tietoa on ainoastaan sijoiteltu lähemmäs sinua kuin muille joutuneille.

Tietoja ei ole vielä saatavilla, mutta oletettavasti voisin tarkistaa tietoa sitten kunhan käynnistyisi tehtäviä tai pyrkimyksiä. (14.5s)

---

`llama3.2:3b` Olen valmis käsittelemään suomenkielistä dataa! Olen koulutettu käsitelleämään erilaisia kieli- ja tekniikkatekijoita, mutta varsinkin suomalaisen kielen ja sen käyttöjärjestelmien ymmärrystä on olemassa vaikeuksia.

Käytän aina kielitietokantoja tai -tietoa esimerkiksi Wikipedia-sta, Valtion tutustumislainausverkkon tiedostoista, tai muuta luottavaa lähteerää. Kun sinulla on suomenkielinen data, olen valmis käsittelemään sitä.

Onko dataa kosketus- tai teksti-tyyppinen? Mikä on tarkoitusta? (28.6s)

---

`llama3:8b` Hei!

Kyseessä on suomalainen chatbot, joka on valmiina vastaanottamaan ja käsittämään suomenkielisiä tilastotietoja. Olen rakennettu oppimaan jaymmärtämään erilaisia kieliä, myös suomea.

Kuten haluat, anna minulle käsiteltäväksi suomenkielistä dataa, ja minä yritän tehdä siitä järkevää. Olkoon tietoa miten hyvin minä kykenen käsittämään sen! (46.2s)

In [22]:
from langchain_community.llms import Ollama
import pandas as pd

# Creating very tiny test sample
df = pd.read_excel('data/Korjuuaikanäytteiden_lisätiedot_16082024.xlsx')
test_sample = df.head(5)

# Function for extracting the values
def extractValuesFromText(llm, df):
    for index, row in df.iterrows():
        text = row['INFOTEKSTI']
        # Define the prompt for the model to extract specific information
        prompt = f"""
        Extract the following information from the text below, the text is in finnish:
        - Maalaji
        - Multavuus
        - pH
        - Nurmen ikä
        - Mitä viljellään

        Text: "{text}"

        Provide the output in this format:
        Maalaji: [extracted maalaji]
        Multavuus: [extracted multavuus]
        ph: [extracted pH]
        Nurmen ikä: [extracted nurmen ikä]
        Mitä viljellään: [extracted mitä viljellään]

        some helpful information:

        Here are all maalaji's:
            "AS   aitosavi",
            "BCt  ruskosammalsaraturve",
            "CSt  sararahkaturve",
            "Ct   saraturve",
            "He   hiue",
            "HeS  hiuesavi",
            "HHk  hieno hiekka",
            "HHt  hieno hieta",
            "HkMr hiekka moreeni",
            "Hs   hiesu",
            "HsMr hiesumoreeni",
            "HsS  hiesusavi",
            "Ht   hieta",
            "HtMr hietamoreeni",
            "HtS  hietasavi",
            "Jm   järvimuta",
            "KHk  karkea hiekka",
            "KHt  karkea hieta",
            "LCt  metsäsaraturve",
            "Lj   lieju",
            "LjS  liejusavi, urpasavi",
            "LSt  metsärahkaturve",
            "Mm   multamaa",
            "Mt   muta",
            "SCt  rahkasaraturve",
            "SMr  savimoreeni",
            "Sr   sora",
            "SrMr soramoreeni",
            "St   rahkaturve"

        These are the different multavuus values:
            vm    vähämultainen  
            m     multava  
            rm    runsasmultainen  
            erm   erittäin runsasmultainen  
            Mm    multamaa  
            
        Multavuus can be close to maalaji written together with it, for example rmHtMr. In this case:
            Maalaji: HtMr Hietamoreeni
            Multavuus: rm Runsasmultainen
            
        Answer to "Mitä viljellään?" is often a list of items with percentages on how much of them is in the field, for example:
            TT Tuure 35,5%, TT Rhonia 24,5%, NN Klaara 15%, RN Karolina 5%, Engl.rh Riikka10%, PA Saija 7,5%, VA Silvester 2,5%
        There can also be just a single value like "Valio Carbo nurmiseos"

        Nurmen ikä is often a number and v or vuosi close to word nurmi, for example 2 v. nurmi. In this case:
            Nurmen ikä: 2v
        """

        # Invoke the model with the prompt
        response = llm.invoke(prompt)

        # Print the result
        print(f"rivi  {int(index) + 1}")
        print("Input text:", text)
        print("Extracted Data:", response)
        print("")


In [23]:
# Define the model you are using
llm = Ollama(model="llama3.2:1b")  # You can choose the model you want

# Run the function
extractValuesFromText(llm, test_sample)

rivi  1
Input text: 3 v. nurmi. 70 kg  N/ha. Ei lietettä eikä täydennyskylvetty.  Valio Carbo nurmiseos.
Extracted Data: Here is the extracted information in the format you requested:

Maalaji: Maalaji
Multavuus: Multavuus
ph: pH
Nurmen ikä: Nurmen ikä
Mitä viljellään: Mitä viljellään

* Maalaji: AS aitosavi, BCt ruskosammalsaraturve, CSt sararahkaturve, Ct saraturve, He hiue, HeS hiuesavi, HHk hieno hiekka, HkMr hiekka moreeni, Hs hiesu, HsMr hiesumoreeni, HsS hiesusavi, Ht hieta, HtMr hietamoreeni, HtS hietasavi, Jm järvimuta, KHk karkea hiekka, KHt karkea hieta, LCt metsäsaraturve, Lj lieju, LjS liejusavi urpasavi, LSt metsärahkaturve, Mm multamaa, Mt muta, SCt rahkasaraturve, SMr savimoreeni, Sr soramoreeni, St rahkaturve

Multavuus: 
* vm vähämultainen
* m multava
* rm runsasmultainen
* erm erittäin runsasmultainen
* Mm multamaa

Nurmen ikä:
* 2v Nurmen ikä

rivi  2
Input text: Koulutila. TT 80%, NN 20%. Lannoitus 14.5. YaraMila Y3 400 kg/ha. Maalaji HHt pH 6,2. 3v. nurmi
Extracte

In [24]:
# Define the model you are using
llm = Ollama(model="llama3.2:3b")  # You can choose the model you want

# Run the function
extractValuesFromText(llm, test_sample)

rivi  1
Input text: 3 v. nurmi. 70 kg  N/ha. Ei lietettä eikä täydennyskylvetty.  Valio Carbo nurmiseos.
Extracted Data: Based on the provided text and information, I was able to extract the following:

Maalaji: Multavuus
Multavuus: rm Runsasmultainen
pH: ei jätettä eikä täydennyskylvetty = pH ei ole näkyvää (ei jätettä eikä täydennyskylvetty on mahdollinen, mutta se ei ole tärkeää tässä tekstissä)
Nurmen ikä: 3 v. nurmi
Mitä viljellään:
TT Tuure 35,5%, TT Rhonia 24,5%, NN Klaara 15%, RN Karolina 5%, Engl.rh Riikka10%, PA Saija 7,5%, VA Silvester 2,5%

rivi  2
Input text: Koulutila. TT 80%, NN 20%. Lannoitus 14.5. YaraMila Y3 400 kg/ha. Maalaji HHt pH 6,2. 3v. nurmi
Extracted Data: Based on the provided text and information, here are the extracted values:

Maalaji: HHt hieno hieta
Multavuus: rm Runsasmultainen (close to mm Mm multamaa)
ph: 6.2
Nurmen ikä: 3v
Mitä viljellään: TT Tuure 20%, NN Klaara 15%

rivi  3
Input text: 2 v. nurmi. HiueS. Ei lietetttä keväällä. Salpietari määrä?  Ca

In [25]:
# Define the model you are using
llm = Ollama(model="llama3:latest")  # You can choose the model you want

# Run the function
extractValuesFromText(llm, test_sample)

rivi  1
Input text: 3 v. nurmi. 70 kg  N/ha. Ei lietettä eikä täydennyskylvetty.  Valio Carbo nurmiseos.
Extracted Data: Here are the extracted information in the requested format:

Maalaji: Not specified (no maalaji code is mentioned in the text)

Multavuus: Not specified (no multavuus value is mentioned in the text)

pH: Not specified (pH value is not mentioned in the text)

Nurmen ikä: 3 v. nurmi

Mitä viljellään: Valio Carbo nurmiseos

rivi  2
Input text: Koulutila. TT 80%, NN 20%. Lannoitus 14.5. YaraMila Y3 400 kg/ha. Maalaji HHt pH 6,2. 3v. nurmi
Extracted Data: Here are the extracted values in the requested format:

Maalaji: HHt
Multavuus: (not mentioned)
ph: 6,2
Nurmen ikä: 3v
Mitä viljellään: (not mentioned)

rivi  3
Input text: 2 v. nurmi. HiueS. Ei lietetttä keväällä. Salpietari määrä?  Carbo 3 niiton seos TT Rubinia 35%+ Uula 15%,  RuoN Karolina 20%, NN Valtteri/Evertti 15% + apiloita
Extracted Data: Here are the extracted information in the format you requested:

Maalaji: